In [5]:
import android_manager
import poke_manager
import win32gui
import win32con
import pyautogui
import time
from ppadb.client import Client as AdbClient
from ppadb.device import Device as AdbDevice
import importlib
import os
from pathlib import Path

In [6]:
importlib.reload(android_manager)
importlib.reload(poke_manager)

#scrcpy로 Device가 실행된 상태
cur_client = AdbClient(host='localhost', port=5037)
#serial number:
#ce10171ab2312a0d04 for s8
#ce071717d4035622047e for note8
pm_s8 = poke_manager.Poke_device(cur_client.device('ce10171ab2312a0d04'))
pm_note8 = poke_manager.Poke_device(cur_client.device('ce071717d4035622047e'))
#my-pm은 둘 중 하나를 복사
# my_pm = pm_s8
my_pm = pm_note8
#always-on-top 옵션 주기 > 나주에는 클래스 안에서 구현. 
pm_devices = [d for d in [pm_s8, pm_note8] if d is not None]
for d in pm_devices:
    x1,y1,x2,y2 = win32gui.GetWindowRect(d.hwnd)
    win32gui.SetWindowPos(d.hwnd, win32con.HWND_TOPMOST, x1, y1, x2-x1, y2-y1, win32con.WM_SHOWWINDOW)
    d.shell('settings put system screen_brightness 0')
print(pm_devices)

Deviced SM-G950N is connedted
Hwnd: 4262080
Deviced SM-N950N is connedted
Hwnd: 1705348
[<poke_manager.Poke_device object at 0x000001F23AF9C810>, <poke_manager.Poke_device object at 0x000001F23A104FD0>]


In [4]:
#선물열기 자동화
profile_xy = (137, 1892)
profile_friend_xy = (755, 163)
my_pm.confidence = 0.95

for i in range(500):
    my_pm.automation_by_image('bcb', profile_xy, new_status='profile')
    my_pm.automation_by_image('profile_me', profile_friend_xy, cur_status='profile', new_status='profile_friend')
    my_pm.automation_by_image('profile_friend', no_click=True, cur_status='profile', new_status='profile_friend')
    my_pm.automation_by_image('present', cur_status='profile_friend', new_status='open_present')
    my_pm.automation_by_image('present_to_open', cur_status='open_present')
    my_pm.automation_by_image('open_cmd', new_status='after_open_present')
    my_pm.automation_by_image('x_green', cur_status='after_open_present', new_status='profile_friend')
    time.sleep(1)

Found bcb at Point(x=1682, y=898)
Img: bcb, T_img: None
Click: (137, 1892)(is_tap_xy: True)
prev status: profile, new status:profile


KeyboardInterrupt: 

In [88]:
#한영 자동전환
keyboard_language_change_xy = (329,1966)
#수원
text = 'tndnjs&vkxmsj vhzptahs0'
#한글로 전환
my_pm.automation_by_image('keyboard_qwerty', device_xy=keyboard_language_change_xy)
my_pm.device.input_text(text)
my_pm.automation_by_image('keyboard_confirm')

Found keyboard_confirm at Point(x=3325, y=1909)
Img: keyboard_confirm, T_img: None
Click: Point(x=3325, y=1909)(is_tap_xy: False)


False

In [97]:
my_pm.status = 'after_exchange_pokemon'

In [3]:
#교환 자동화
first_pokemon_in_list = (191, 734)
exchange_special_change_no = (523, 1302)
while True:
    my_pm.automation_by_image('exchange_cmd', new_status='before_exchange')
    my_pm.automation_by_image('exchange_pokemon_list', device_xy=first_pokemon_in_list)
    my_pm.automation_by_image('exchange_next_cmd')
    my_pm.automation_by_image('exchange_100_cost', target_img='exchange_confirm', new_status='after_exchange')
    my_pm.automation_by_image('exchange_special_change', exchange_special_change_no)
    my_pm.automation_by_image('x_green', cur_status='after_exchange', new_status='before_exchange')

Found exchange_pokemon_list at Point(x=3621, y=1133)
Img: exchange_pokemon_list, T_img: None
Click: (191, 734)(is_tap_xy: True)
Found exchange_next_cmd at Point(x=3619, y=1789)
Img: exchange_next_cmd, T_img: None
Click: Point(x=3619, y=1789)(is_tap_xy: False)
Found exchange_cmd at Point(x=3776, y=1754)
Img: exchange_cmd, T_img: None
Click: Point(x=3776, y=1754)(is_tap_xy: False)
prev status: initial, new status:before_exchange
Found exchange_pokemon_list at Point(x=3621, y=1133)
Img: exchange_pokemon_list, T_img: None
Click: (191, 734)(is_tap_xy: True)
Found exchange_next_cmd at Point(x=3619, y=1789)
Img: exchange_next_cmd, T_img: None
Click: Point(x=3619, y=1789)(is_tap_xy: False)
Found exchange_100_cost at Point(x=3531, y=1518)
Img: exchange_100_cost, T_img: exchange_confirm
Click: Point(x=3436, y=1521)(is_tap_xy: False)
prev status: before_exchange, new status:after_exchange
Found x_green at Point(x=3624, y=1887)
Img: x_green, T_img: None
Click: Point(x=3624, y=1887)(is_tap_xy: Fals

error: (1400, 'GetWindowRect', '잘못된 창 핸들입니다.')

In [73]:
# my_pm.click_xy((3132, 1887))
my_pm.automation_by_image('x_green')

Found x_green at Point(x=3132, y=1887)
Img: x_green, T_img: None
Click: Point(x=3132, y=1887)(is_tap_xy: False)


True

In [ ]:
import subprocess
subprocess.run()
pyautogui.displayMousePosition()

In [102]:
#언어 입력 관련
text = 'tjdnf&!vkxmsjvhzptahs0'
pm_note8.device.input_text(text)

''